<a href="https://colab.research.google.com/github/szn5400/CO2_all/blob/main/PINN_trial_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
X = [[2,2],[3,3],[4,4],[5,5],[6,6],[7,7]]
#Y = [[4,4],[6,6],[8,8],[10,10],[12,12],[14,14]]
Y = [[4,4],[9,9],[16,16],[25,25],[36,36],[49,49]]

In [ ]:
size_input = 2
size_hidden = 2
size_output = 2
number_of_train_examples = 6
number_of_test_examples = 0

In [ ]:
import os
import numpy as np
import time
import tensorflow as tf

In [ ]:
np.random.seed(5)
tf.random.set_seed(5)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_probability as tfp
import scipy
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
initializer = tf.keras.initializers.GlorotNormal()

inputs = layers.Input(shape=(2,), dtype=tf.float64)
#z = layers.Lambda(lambda X: 2.0*(X - lb)/(ub - lb) - 1.0)(inputs)
z = layers.Dense(2, activation="tanh", kernel_initializer=initializer)(inputs)
z = layers.Dense(2, activation="tanh", kernel_initializer=initializer)(z)

outputs = layers.Dense(2, kernel_initializer=initializer)(z)

U_model_naive = tf.keras.Model(inputs=inputs, outputs=outputs)

U_model_naive.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 6         
                                                                 
 dense_4 (Dense)             (None, 2)                 6         
                                                                 
 dense_5 (Dense)             (None, 2)                 6         
                                                                 
Total params: 18
Trainable params: 18
Non-trainable params: 0
_________________________________________________________________


In [ ]:
BATCH_SIZE = 6
EPOCHS = 1000

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01,
                                     beta_1=0.99,
                                     epsilon=1e-1)


In [ ]:
# @tf.function
def train_step(X_u, u_true):
    with tf.GradientTape() as tape:
        print(X_u)
        X_u = tf.convert_to_tensor(X_u)
        print(X_u)
        #X_u = X_u.flatten()
        u_pred = tf.cast(U_model_naive(X_u), tf.float64)

        loss = tf.losses.MeanSquaredError()(u_true, u_pred)
    grads = tape.gradient(loss, U_model_naive.trainable_weights)
    optimizer.apply_gradients(zip(grads, U_model_naive.trainable_weights))
    return loss

loss_hist = []
for epoch in range(EPOCHS):
    ep_loss = []
    #loss_i = train_step(X, Y)
    #ep_loss.append(loss_i)
    loss_val = 0
    for i in range(len(X)):
      loss_i = train_step(X[i], Y[i])
      loss_val += loss_i
    ep_loss.append(loss_val)
    '''
    for step, (X_u_batch_train, u_batch_train) in enumerate(U_dataset):

        loss_i = train_step(X_u_batch_train, u_batch_train)
        ep_loss.append(loss_i)
    '''
    loss_hist.append(tf.reduce_mean(ep_loss))

    #U_dataset = tf.data.Dataset.from_tensor_slices((X_u_train, u_train))
    #U_dataset = U_dataset.shuffle(buffer_size=100).batch(BATCH_SIZE)

    if epoch % 10 == 0:
        print(f"Epoch {epoch} Loss: {tf.reduce_mean(ep_loss)}")

[2, 2]
tf.Tensor([2 2], shape=(2,), dtype=int32)


ValueError: ignored